In [1]:
from utilities.utils import fullpath
import pandas as pd
import numpy as np

In [2]:
path_weight_caps = fullpath("data","strategy","caps","20250331.csv")
path_weight_msr =  fullpath("data","strategy","msr","20250331.csv")
path_prices = fullpath("data","market","prices","20250331.csv")

In [3]:
df_weights_caps = pd.read_csv(path_weight_caps)
df_weights_msr = pd.read_csv(path_weight_msr)
df_prices = pd.read_csv(path_prices)

In [4]:
df_prices["Date"] = pd.to_datetime(df_prices["Date"])

In [5]:
start_date,end_date = df_prices.iloc[[0,-1]]['Date'].values
all_dates = pd.date_range(start=start_date, end=end_date)
df_prices_all_dates = pd.DataFrame(all_dates, columns=["Dates"])

In [6]:
df_data_market_all = pd.merge(df_prices, df_prices_all_dates, left_on="Date", right_on="Dates", how="right")
df_data_market_all.drop("Date", axis=1, inplace=True)
df_data_market_all.ffill(axis=0,inplace=True)
df_data_market_all

# get month ends
idx = df_data_market_all.Dates.dt.is_month_end
df_data_market_all = df_data_market_all.loc[idx]
df_data_market_all.drop("Dates", axis=1, inplace=True)

df_data_market_all

In [7]:
# get returns from prices
df_returns = df_data_market_all.pct_change()
df_expected_returns = df_returns.mean()
df_expected_returns

In [8]:
df_expected_returns = pd.DataFrame(df_expected_returns, columns = ["ExpectedReturns"])
df_expected_returns["Symbol"] = df_expected_returns.index
df_expected_returns = df_expected_returns.reset_index(drop=True)
df_expected_returns

In [9]:
df_covariance_matrix = df_data_market_all.cov() * 12
df_covariance_matrix 

In [10]:
df_ptf = pd.merge(df_weights_caps, df_expected_returns, on="Symbol")
df_ptf.rename(columns={"Weights": "Weights_CW"}, inplace=True)
df_ptf

In [11]:
df_weights_msr.rename(columns={"Weights":"Weights_MSR"}, inplace=True)

In [12]:
df_ptf = pd.merge(df_weights_msr, df_ptf, on="Symbol")
df_ptf

In [13]:
# Move 'ExpectedReturns' to the second column
columns = list(df_ptf.columns)
columns.remove("ExpectedReturns")
columns.insert(1, "ExpectedReturns")  # Insert at the second position
df_ptf = df_ptf[columns]

# Display the updated DataFrame
df_ptf

In [14]:
# ptf_return = np.dot(df_ptf_caps.Weights, df_ptf_caps.ExpectedReturns)
# ptf_volatility = np.dot(df_ptf_caps.Weights.T, np.dot(df_covariance_matrix, df_ptf_caps.Weights))
# ptf_return, ptf_volatility**(1/2)

In [15]:
# ptf_return = np.dot(df_ptf_msr.Weights, df_ptf_msr.ExpectedReturns)
# ptf_volatility = np.dot(df_ptf_msr.Weights.T, np.dot(df_covariance_matrix, df_ptf_msr.Weights))
# ptf_return, ptf_volatility**(1/2)

In [16]:
np.random.seed(42)
weights_rand = np.random.random(size=(40,100000))

In [17]:
rands_sum = sum(weights_rand)

In [18]:
weights_rand_normalized = weights_rand/rands_sum
sum(sum(weights_rand_normalized))

In [19]:
def get_ptf_returns_volatility(weights):
    ptf_return = np.dot(df_ptf_msr.Weights, df_ptf_msr.ExpectedReturns)
    ptf_volatility = np.dot(df_ptf_msr.Weights.T, np.dot(df_covariance_matrix, df_ptf_msr.Weights)) ** (1/2)
    return ptf_return, ptf_volatility

In [20]:
# Create column names for the new weights
new_columns = [f"Weights_Rand_{i}" for i in range(100000)]

# Add the weights_rand_normalized as new columns to df_ptf
df_weights_rand = pd.DataFrame(weights_rand_normalized, columns=new_columns)  # Transpose to match rows

# Concatenate the new columns with df_ptf
df_ptf = pd.concat([df_ptf.reset_index(drop=True), df_weights_rand.reset_index(drop=True)], axis=1)

# Display the updated DataFrame
df_ptf

In [21]:
df_ptf.iloc[:,2:].sum().sum()

In [22]:
ptf_returns = df_ptf.iloc[:,2:].mul(df_ptf["ExpectedReturns"], axis=0)
df_ptf_returns = np.sum(ptf_returns, axis=0)
df_ptf_returns

In [23]:
# df_ptf.to_csv("test.csv", index=False)

In [24]:
# Assuming df_covariance_matrix is your covariance matrix
# Extract only the weight columns from df_ptf
weight_columns = [col for col in df_ptf.columns if col.startswith("Weights")]

# Initialize a dictionary to store results
portfolio_volatilities = {}

# Loop through each weight column
for col in weight_columns:
    # Extract the weight vector
    weights = df_ptf[col].values
    
    # Compute w * cov_matrix * w^T
    portfolio_volatility = (np.dot(weights.T, np.dot(df_covariance_matrix, weights)))**(1/2)
    
    # Store the result
    portfolio_volatilities[col] = portfolio_volatility

# Convert results to a DataFrame for better visualization
df_portfolio_volatilities = pd.DataFrame.from_dict(portfolio_volatilities, orient="index", columns=["Volatility"])

# Display the results
print(df_portfolio_volatilities)

In [25]:
import matplotlib.pyplot as plt

# Assuming df_ptf_returns and df_portfolio_volatilities are aligned
# Extract returns and volatilities
returns = df_ptf_returns.values
volatilities = df_portfolio_volatilities["Volatility"].values

# Plot the efficient frontier
plt.figure(figsize=(10, 6))
plt.plot(volatilities, returns, 'o', label="Efficient Frontier", color="blue", alpha=0.7)

# Highlight MSR (column 1) in green
plt.scatter(volatilities[0], returns[0], color="green", label="MSR (Column 1)", s=100)

# Highlight CW (column 2) in red
plt.scatter(volatilities[1], returns[1], color="red", label="CW (Column 2)", s=100)

# Add labels and legend
plt.title("Efficient Risk Frontier", fontsize=16)
plt.xlabel("Portfolio Volatility (Risk)", fontsize=12)
plt.ylabel("Portfolio Return", fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()

In [26]:
weight_columns = [col for col in df_ptf.columns if col.startswith("Weights")]
df_ptf[weight_columns] = df_ptf[weight_columns].div(df_ptf[weight_columns].sum(axis=0), axis=1)

ptf_returns = df_ptf.iloc[:, 2:].mul(df_ptf["ExpectedReturns"], axis=0)
df_ptf_returns = ptf_returns.sum(axis=0)

portfolio_volatilities = {}
for col in weight_columns:
    weights = df_ptf[col].values
    portfolio_volatility = np.dot(weights.T, np.dot(df_covariance_matrix, weights)) ** 0.5
    portfolio_volatilities[col] = portfolio_volatility

df_portfolio_volatilities = pd.DataFrame.from_dict(portfolio_volatilities, orient="index", columns=["Volatility"])

import matplotlib.pyplot as plt

# Extract returns and volatilities
returns = df_ptf_returns.values
volatilities = df_portfolio_volatilities["Volatility"].values

# Plot the efficient frontier
plt.figure(figsize=(10, 6))
plt.plot(volatilities, returns, 'o-', label="Efficient Frontier", color="blue", alpha=0.7)

# Highlight MSR (column 1) in green
plt.scatter(volatilities[0], returns[0], color="green", label="MSR (Column 1)", s=100)

# Highlight CW (column 2) in red
plt.scatter(volatilities[1], returns[1], color="red", label="CW (Column 2)", s=100)

# Add labels and legend
plt.title("Efficient Risk Frontier", fontsize=16)
plt.xlabel("Portfolio Volatility (Risk)", fontsize=12)
plt.ylabel("Portfolio Return", fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()